In [5]:
import numpy as np
import pandas as pd
import time
import csv
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import itertools
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import nltk

import networkx as nx
import matplotlib.pyplot as plt
import sumy

In [6]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer

In [29]:

input_path='F:/Course/2021 Fall/ISE 540/Project/book/history' # book directory
book_list=os.listdir(input_path)                              # book list

In [30]:
result=pd.DataFrame(columns=['tf_sum','tf_len','optimal_cut'])

In [ ]:

done=[]
wrong=[]
for books in book_list: 
    try:
        input_file=input_path+'/'+books

        def read_article(file_name):
            file = open(file_name, "r", encoding="UTF-8")
            filedata = file.readlines()
            # print(filedata)

            filedata_new = " ".join(filedata)               # join every elements in the list into a single element
            paragraph = filedata_new.replace("\n \n ", ".") # replace newlines between paragraphs by periods(there are usually 2 or 3 newlines between paragraphs, so replace by at least 2 newlines)
            paragraph = paragraph.split(".")                # Split all periods, so we get sentence by sentence
            # print(paragraph)

            sentences = []
            for sentence in paragraph:
                sentence = sentence.replace("\n", "") # remove the extra newlines between sentences
                # print(sentence)
                sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))

            sentences = [[string for string in sublist if string] for sublist in sentences] # Remove the empty elements in every sentences if any
            sentences = [string for string in sentences if string] # remove empty lists if any
            # sentences.pop() # Drop the last two empty elements
            end=0
            for test in sentences:
                if ('***END' in test) or (('END' in test)&('***' in test)):
                    break
                end=end+1

            if end!=len(sentences):
                del sentences[end:]

            start=0
            for test in sentences:
                if ('***START' in test )or (('START' in test)&('***' in test)):
                    break
                start=start+1
            if start!=len(sentences): 
                del sentences[:start+1]

            return sentences

        book=read_article(input_file)

        def split(input_book):                   # chapterize the book

            cnt=0
            idx=dict()

            for i in input_book:
                if ('chapter' in map(str.lower, i[:1])) & (len(i)>=2) :
                    idx[tuple(i)]=cnt

                elif ('chapitre' in map(str.lower, i[:1])) & (len(i)>=2) :
                    idx[tuple(i)]=cnt   

                elif ('chap' in map(str.lower, i[:1])):
                    idx[tuple(i)]=cnt

                elif ('section' in map(str.lower, i[:1])):
                    idx[tuple(i)]=cnt    

                cnt=cnt+1

            page=sorted(idx.values())

            chapter=dict()
            cnt=0
            while cnt <=len(page)-1:

                if cnt!=len(page)-1:
                    chapter[cnt+1]=input_book[page[cnt]:page[cnt+1]]

                else:
                    chapter[cnt+1]=input_book[page[cnt]:]
                cnt=cnt+1
            return chapter

        chap=split(book)

        if chap =={}:                                 # backup chapterize method if split function fail
            point=1
            for page in range(300,len(book),300):
                if (page+300)>len(book):
                    chap[point]=book[page-300:page]
                    chap[point+1]=book[page:]
                else:
                    chap[point]=book[page-300:page]
                point=point+1

        def chap_string(input_para):           # convert chap into string
            string=[]
            for tok in input_para:
                string.append(" ".join(tok))
            string='. '.join(string)
            return(string)

        def agg_sum(input_sum):                    # convert individual output sentences from Sumy into one-paragraph summary
            in_sum=[]
            for sens in input_sum:
                in_sum.append(str(sens))
            in_sum=' '.join(in_sum)
            return(in_sum)


        plots=dict()

        tf_sum=dict()
        tf_len=dict()
        for i in chap:

            scores=[]

            chap_sum=[]
            for j in chap[i]:
                chap_sum.append(" ".join(j))
            chap_sum=' '.join(chap_sum) 

            if len(chap[i])>500 :
                continue
            for k in range(1,min(30,len(chap[i])),1):       



                parser = PlaintextParser.from_string(chap_string(chap[i]),Tokenizer("english"))
                summarizer = LexRankSummarizer()           
                summary = summarizer(parser.document,k)      # find top 30 important sentence
 
                data=[agg_sum(summary), chap_sum]            #record tf-idf info
                vectorizer = TfidfVectorizer( )
                vector1 = vectorizer.fit_transform(data)
                tf_sum[i]=sum(vector1.toarray()[1])
                tf_len[i]=len(vector1.toarray()[1])
                sim=cosine_similarity(vector1)[1][0]
                scores.append(sim)
            plots[i]=scores.copy()
 
        cut=dict()                                          # choose optimal cut
        for i in plots: 
            pos=len(plots[i])
            cut[i]=1
            for sim in plots[i][::-1]:
                if sim<=0.8*max(plots[i]):
                    cut[i]=pos
                    break
                pos=pos-1

        for indx in tf_sum:
            df2 = {'tf_sum': tf_sum[indx], 'tf_len': tf_len[indx], 'optimal_cut': cut[indx]}
            result = result.append(df2, ignore_index = True)
        done.append(books)
    except:
        wrong.append(books)
        continue
        